In [1]:
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage

## Building out the Dataframes

In [2]:
SQL = '''SELECT * from Transfers247'''
df_247 = (fx.connDBAndReturnDF(SQL)).set_index('IDYR')
df_247.index.name = '247_IDYR'
df_247

,ID,PlayerName,Year,StandardizedPosition,KeyPositionGroup
247_IDYR,,,,,
paulthurston_nebraska_2012,paulthurston_nebraska,paulthurston,2012,OL,1
gregmcmullen_nebraska_2012,gregmcmullen_nebraska,gregmcmullen,2012,DE,2
jordanwesterkamp_nebraska_2012,jordanwesterkamp_nebraska,jordanwesterkamp,2012,WR,1
imanicross_nebraska_2012,imanicross_nebraska,imanicross,2012,RB,1
tommyarmstrong_nebraska_2012,tommyarmstrong_nebraska,tommyarmstrong,2012,QB,1
...,...,...,...,...,...
dondrellharris_troy_2013,dondrellharris_troy,dondrellharris,2013,QB,1
jordanchunn_troy_2013,jordanchunn_troy,jordanchunn,2013,FB,1
bryanslater_troy_2013,bryanslater_troy,bryanslater,2013,DE,2


In [3]:
SQL = '''SELECT * from UnlinkedNCAA'''
df_ncaa = (fx.connDBAndReturnDF(SQL)).set_index('ID')
df_ncaa.index.name = 'NCAA_ID'
df_ncaa

,PlayerName,Year,StandardizedPosition,KeyPositionGroup
NCAA_ID,,,,
aadarkwah_massachusetts,aadarkwah,2017,WR,1
aadreekisconner_mississippistate,aadreekisconner,2019,DB,2
aarenvaughns_utahstate,aarenvaughns,2017,WR,1
aarionharvey_centralmichigan,aarionharvey,2020,OL,1
aaronabbott_easternmichigan,aaronabbott,2015,DB,2
...,...,...,...,...
zionjohnson_bostoncollege,zionjohnson,2019,OL,1
zionjohnson_louisville,zionjohnson,2019,OL,1
zionkelly_airforce,zionkelly,2020,DB,2


In [4]:
indexer = recordlinkage.Index()
indexer = recordlinkage.BlockIndex(on=['KeyPositionGroup'])


In [5]:
pairs = indexer.index(df_247, df_ncaa)

In [6]:
from functions import YearNFL

sumFields = []
c = recordlinkage.Compare()
c.string('PlayerName', 'PlayerName', method='damerau_levenshtein', label='PlayerName')
sumFields.append('PlayerName')
c.exact('StandardizedPosition', 'StandardizedPosition', label='StandardizedPosition')
sumFields.append('StandardizedPosition')
c.exact('KeyPositionGroup', 'KeyPositionGroup', label='KeyPositionGroup')
sumFields.append('KeyPositionGroup')
c.add(YearNFL('Year', 'Year', label='Year'))
sumFields.append('Year')




In [7]:
features = c.compute(pairs, df_247, df_ncaa)

In [8]:
sum = 0
for field in sumFields:
    sum = sum + features[field]

features['sum'] = sum / len(sumFields)

features = features[features['Year'] == 1.0]

features

PlayerName  \
247_IDYR                  NCAA_ID                                  
sampsonjames_indiana_2019 aadarkwah_massachusetts       0.166667   
                          aarenvaughns_utahstate        0.166667   
                          aaronaiken_coastalcarolina    0.333333   
                          aaronamaama_utah              0.250000   
                          aaronbennett_kansasstate      0.166667   
...                                                          ...   
jackmartin_troy_2019      zachrobinson_texasstate       0.250000   
                          zachvonrosenberg_lsu          0.250000   
                          zachwood_cincinnati           0.200000   
                          zackcimaglia_southcarolina    0.500000   
                          zackrogers_newmexico          0.300000   

                                                      StandardizedPosition  \
247_IDYR                  NCAA_ID                                            
sampsonjames_indiana_2019 aadarkwah_massachusetts                        0   
                          aarenvaughns_utahstate                         0   
                          aaronaiken_coastalcarolina                     0   
                          aaronamaama_utah                               0   
                          aaronbennett_kansasstate                       0   
...                                                                    ...   
jackmartin_troy_2019      zachrobinson_texasstate                        0   
                          zachvonrosenberg_lsu                           0   
                          zachwood_cincinnati                            0   
                          zackcimaglia_southcarolina                     1   
                          zackrogers_newmexico                           1   

                                                      KeyPositionGroup  Year  \
247_IDYR                  NCAA_ID                                              
sampsonjames_indiana_2019 aadarkwah_massachusetts                    1   1.0   
                          aarenvaughns_utahstate                     1   1.0   
                          aaronaiken_coastalcarolina                 1   1.0   
                          aaronamaama_utah                           1   1.0   
                          aaronbennett_kansasstate                   1   1.0   
...                                                                ...   ...   
jackmartin_troy_2019      zachrobinson_texasstate                    1   1.0   
                          zachvonrosenberg_lsu                       1   1.0   
                          zachwood_cincinnati                        1   1.0   
                          zackcimaglia_southcarolina                 1   1.0   
                          zackrogers_newmexico                       1   1.0   

                                                           sum  
247_IDYR                  NCAA_ID                               
sampsonjames_indiana_2019 aadarkwah_massachusetts     0.541667  
                          aarenvaughns_utahstate      0.541667  
                          aaronaiken_coastalcarolina  0.583333  
                          aaronamaama_utah            0.562500  
                          aaronbennett_kansasstate    0.541667  
...                                                        ...  
jackmartin_troy_2019      zachrobinson_texasstate     0.562500  
                          zachvonrosenberg_lsu        0.562500  
                          zachwood_cincinnati         0.550000  
                          zackcimaglia_southcarolina  0.875000  
                          zackrogers_newmexico        0.825000  

[39597674 rows x 5 columns]

In [11]:
#features.insert(0, 'sourceID', features.index.get_level_values(0))
#features.insert(1, 'targetID', features.index.get_level_values(1))

filteredList = []
for idx, data in features.groupby(level=0):
    data = data.loc[data['sum'].idxmax()]
    if (data['sum'] > .8):
        filteredList.append(data)
dfFinal = pd.DataFrame()
dfFinal = dfFinal.append(filteredList)
dfFinal.to_csv("resultsNCAATransfer.csv")